`https://python.langchain.com/docs/tutorials/rag/`

In [1]:
from langchain.schema import Document   # whichever Document import you use
import gzip, pickle
from langchain_text_splitters import RecursiveCharacterTextSplitter
from helper.academicCloudEmbeddings import AcademicCloudEmbeddings
from langchain.vectorstores import FAISS
import streamlit as st
from langchain.retrievers import ParentDocumentRetriever
from langchain.docstore import InMemoryDocstore

# Indexing
## 1. Load the data
*In our case the data needs to be crawled first. See `crawl.ipynb`. There we are storing the documents in a pickle file we can load here*

In [2]:
with gzip.open("docs.pkl.gz", "rb") as f:
    docs = pickle.load(f)

# 2.  give every parent document a stable id  (one-liner)
for i, d in enumerate(docs):
    d.metadata["doc_id"] = f"doc_{i}"            # <- save this id

## 2. Split the loaded data
We are splitting large documents into smaller chunks for indexing the data and passing it into a model. Large chunks would be worse for search

In [3]:
# splitten – jede URL bleibt als metadata erhalten
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],
)
child_docs = splitter.split_documents(docs)

## 3. Store
We are storing the created chunks:
1. creating embeddings using the GWDG model
2. storing in a FAISS store which can be saved locally to use later. That way we don't need to create the store every time we want to start the app

In [4]:
# Embeddings und FAISS
embedder = AcademicCloudEmbeddings(
    api_key=st.secrets["GWDG_API_KEY"],
    url=st.secrets["BASE_URL_EMBEDDINGS"],
)
child_vs = FAISS.from_documents(child_docs, embedder)
child_vs.save_local("faiss_child_index")

In [10]:
from langchain_core.stores import InMemoryStore

# store the *parents* in an in-memory doc-store and pickle it
parent_store = InMemoryStore()
parent_store.mset([(d.metadata["doc_id"], d) for d in docs])

with open("parent_store.pkl", "wb") as f:
    pickle.dump(parent_store, f)

In [14]:
# test
with open("parent_store.pkl", "rb") as f:
    parent_store = pickle.load(f)

retriever = ParentDocumentRetriever(
    vectorstore    = child_vs,   # FAISS with your child chunks
    docstore       = parent_store,   # ✔︎ now a BaseStore subclass
    child_splitter = splitter,
    search_kwargs  = {"k": 4},
)
print(retriever.get_relevant_documents("Eduroam Iphone")[0].page_content)

support:wlan:eduroam_iphone_ipad
# eduroam mit iPhone oder iPad (ab Version iOS 11 und iPadOS 11)
Im Folgenden wird erklärt, wie Du Dein iPhone oder iPad mit dem WPA2 verschlüsselten Netzwerk eduroam verbinden kannst. Vorweg das Wichtigste: 
Bei der Einrichtung der Verbindung bitte beachten: Der **„Benutzername“** ist eure **studentische Email-Adresse**! Also bspw. test.test@stud.uni-goettingen.de 
Seit kurzem werden ältere Sicherheitsprotokolle nicht mehr unterstützt. Um eine erfolgreiche Verbindung mit eduroam aufzubauen, wird _mindestens_ iOS Version 11 oder iPadOs Version 11 auf dem jeweiligen Geräte benötigt. Dafür unter Einstellungen–>Allgemein–>Software Update nach neuen Updates suchen.   

Bitte nutze Safari als Browser, um eine erfolgreiche Installation und Einrichtung zu gewährleistet.   

## eduroam
Du kannst dich mit deinem iPhone oder iPad in das WLAN der Universität Göttingen (eduroam) einwählen und so mit sehr hohen Datenraten surfen. 
Wir empfehlen die Nutzung des versc